In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
def isskin(image):
    h_min = 0
    h_max = 128
    s_min = 100
    s_max = 150
    v_min = 0
    v_max = 128

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_hsv = np.array([h_min, s_min, v_min])
    upper_hsv = np.array([h_max, s_max, v_max])

    skinMask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skinMask = cv2.erode(skinMask, kernel, iterations=2)
    skinMask = cv2.dilate(skinMask, kernel, iterations=2)

    skinMask = cv2.GaussianBlur(skinMask, (5, 5), 0)

    skin = cv2.bitwise_and(image, image, mask=skinMask)

    # alpha = np.uint8(skinMask > 0) * 255
    # result = cv2.merge((skin, alpha))

    return skin

In [3]:
def apply_gamma_correction(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

In [4]:
# Skin Segmentation
def extract_skin_and_preprocess(img):
    # skin_img = isskin(img)

    # if skin_img.shape[2] == 4:
    #     skin_img = skin_img[:, :, :3]
        
    corrected = apply_gamma_correction(img)
    # corrected = apply_gamma_correction(skin_img)
    denoised = cv2.fastNlMeansDenoising(corrected, h=10)
    # denoised = cv2.fastNlMeansDenoising(skin_img, h=10)
    img_resized = cv2.resize(denoised, (128, 128))

    # Convert to grayscale
    # gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    # return gray
    return img_resized

In [5]:
def load_data(dataset_path):
    classes = {'Acne': 0, 'Bags': 1, 'Redness': 2}
    X, y = [], []

    for class_name, label in classes.items():
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_path):
            print(f"Folder not found: {class_path}")
            continue
        for subfolder in os.listdir(class_path):
            subfolder_path = os.path.join(class_path, subfolder)
            if os.path.isdir(subfolder_path):
                for img_name in os.listdir(subfolder_path):
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = extract_skin_and_preprocess(img)
                        X.append(img)
                        y.append(label)
                    else:
                        print(f"Failed to load image: {img_path}")

    print(f"Loaded {len(X)} images from {len(classes)} classes.")
    return np.array(X), np.array(y)

In [6]:
# Load data
dataset_path = 'Dataset'
X, y = load_data(dataset_path)

Loaded 471 images from 3 classes.


In [7]:
# LBP Feature Extraction
def extract_lbp_features(img, P=8, R=1):
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if len(img.shape) == 3:  # If the image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img  # Already grayscale
    lbp = local_binary_pattern(gray, P=P, R=R, method='uniform')
    hist, _ = np.histogram(lbp, bins=np.arange(0, P+3), range=(0, P+2))
    hist = hist.astype('float')
    hist /= (hist.sum() + 1e-6)
    return hist

# HOG Feature Extraction
def extract_hog_features(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if len(img.shape) == 3:  # If the image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    features, _ = hog(gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=True)
    return features

# Color Histogram Feature Extraction
def extract_color_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    # bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    if len(img.shape) != 3 or img.shape[2] < 3:  # Ensure the image has 3 channels
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [256], h_range)
    hist_s = cv2.calcHist([hsv], [1], None, [256], s_range)
    hist_v = cv2.calcHist([hsv], [2], None, [256], v_range)
    hist_h /= hist_h.sum() + 1e-6
    hist_s /= hist_s.sum() + 1e-6
    hist_v /= hist_v.sum() + 1e-6
    return np.concatenate([hist_h.flatten(), hist_s.flatten(), hist_v.flatten()])

# GLCM Feature Extraction
def extract_glcm_features(img, distances=[5], angles=[0], properties=('contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation')):
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if len(img.shape) == 3:  # If the image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img  # Already grayscale
    glcm = graycomatrix(gray, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    glcm_features = []
    for prop in properties:
        glcm_features.append(graycoprops(glcm, prop).flatten())
    return np.concatenate(glcm_features)

# Combined Feature Extraction
def extract_combined_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    lbp_features = extract_lbp_features(img)
    hog_features = extract_hog_features(img)
    color_features = extract_color_features(img, 
                                            h_range, s_range, v_range)
    glcm_features = extract_glcm_features(img)
    return np.concatenate([lbp_features, hog_features, color_features, glcm_features])
    # return hog_features


## HOG FEATURE EXTRACTION ONLY

In [8]:
X_hog = np.array([extract_hog_features(img) for img in X])
X_train_hog, X_val_hog, y_train_hog, y_val_hog = train_test_split(X_hog, y, test_size=0.2, random_state=42)

In [9]:
### TRAIN NON STACKING MODELS

model_xgb_hog = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
model_lr_hog = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_svm_hog = SVC(kernel='linear', probability=True)
model_rf_hog = RandomForestClassifier(n_estimators=50, random_state=42)

# Stacking Classifier
model_lr_hog.fit(X_train_hog, y_train_hog)
model_svm_hog.fit(X_train_hog, y_train_hog)
model_rf_hog.fit(X_train_hog, y_train_hog)
model_xgb_hog.fit(X_train_hog, y_train_hog)

# Evaluate model
y_pred_lr_no_hog = model_lr_hog.predict(X_val_hog)
y_pred_svm_no_hog = model_svm_hog.predict(X_val_hog)
y_pred_rf_no_hog = model_rf_hog.predict(X_val_hog)
y_pred_xgb_no_hog = model_xgb_hog.predict(X_val_hog)

c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:55:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [10]:
### NON STACKING HOG ONLY MODELS ACCURACY

print(f"Validation Accuracy (HOG Only) With Logistic Regression With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_lr_no_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_lr_no_hog))
print(f"Validation Accuracy (HOG Only) With Support Vector Machine With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_svm_no_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_svm_no_hog))
print(f"Validation Accuracy (HOG Only) With Random Forest With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_rf_no_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_rf_no_hog))
print(f"Validation Accuracy (HOG Only) With XGBoosting With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_xgb_no_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_xgb_no_hog))

Validation Accuracy (HOG Only) With Logistic Regression With Gamma Correction:
Accuracy: 66.32%
              precision    recall  f1-score   support

           0       0.57      0.55      0.56        31
           1       0.74      0.79      0.76        33
           2       0.67      0.65      0.66        31

    accuracy                           0.66        95
   macro avg       0.66      0.66      0.66        95
weighted avg       0.66      0.66      0.66        95

Validation Accuracy (HOG Only) With Support Vector Machine With Gamma Correction:
Accuracy: 67.37%
              precision    recall  f1-score   support

           0       0.59      0.55      0.57        31
           1       0.73      0.82      0.77        33
           2       0.69      0.65      0.67        31

    accuracy                           0.67        95
   macro avg       0.67      0.67      0.67        95
weighted avg       0.67      0.67      0.67        95

Validation Accuracy (HOG Only) With Random 

In [11]:
### TRAIN HOG ONLY STACKING MODELS

# Train base models
base_models = [
    ('svm', SVC(kernel='linear', probability=True)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
]
# Meta-model
meta_model_lr_hog = LogisticRegression(multi_class='multinomial', solver='lbfgs')
meta_model_svm_hog = SVC(kernel='linear', probability=True)
meta_model_rf_hog = RandomForestClassifier(n_estimators=50, random_state=42)
meta_model_xgb_hog = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Stacking Classifier
stacking_lr_hog = StackingClassifier(estimators=base_models, final_estimator=meta_model_lr_hog, cv=5)
stacking_svm_hog = StackingClassifier(estimators=base_models, final_estimator=meta_model_svm_hog, cv=5)
stacking_rf_hog = StackingClassifier(estimators=base_models, final_estimator=meta_model_rf_hog, cv=5)
stacking_xgb_hog = StackingClassifier(estimators=base_models, final_estimator=meta_model_xgb_hog, cv=5)
stacking_lr_hog.fit(X_train_hog, y_train_hog)
stacking_svm_hog.fit(X_train_hog, y_train_hog)
stacking_rf_hog.fit(X_train_hog, y_train_hog)
stacking_xgb_hog.fit(X_train_hog, y_train_hog)

# Evaluate model
y_pred_lr_hog = stacking_lr_hog.predict(X_val_hog)
y_pred_svm_hog = stacking_svm_hog.predict(X_val_hog)
y_pred_rf_hog = stacking_rf_hog.predict(X_val_hog)
y_pred_xgb_hog = stacking_xgb_hog.predict(X_val_hog)

c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:56:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:56:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:57:15] WARNING: C:\buildki

In [12]:
### HOG ONLY STACKING MODELS ACCURACY

print(f"Validation Accuracy (HOG Only) Stacking With Logistic Regression With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_lr_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_lr_hog))
print(f"Validation Accuracy (HOG Only) Stacking With Support Vector Machine With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_svm_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_svm_hog))
print(f"Validation Accuracy (HOG Only) Stacking With Random Forest With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_rf_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_rf_hog))
print(f"Validation Accuracy (HOG Only) Stacking With XGBoosting With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val_hog, y_pred_xgb_hog) * 100:.2f}%")
print(classification_report(y_val_hog, y_pred_xgb_hog))

Validation Accuracy (HOG Only) Stacking With Logistic Regression With Gamma Correction:
Accuracy: 67.37%
              precision    recall  f1-score   support

           0       0.59      0.52      0.55        31
           1       0.74      0.79      0.76        33
           2       0.67      0.71      0.69        31

    accuracy                           0.67        95
   macro avg       0.67      0.67      0.67        95
weighted avg       0.67      0.67      0.67        95

Validation Accuracy (HOG Only) Stacking With Support Vector Machine With Gamma Correction:
Accuracy: 63.16%
              precision    recall  f1-score   support

           0       0.53      0.58      0.55        31
           1       0.74      0.76      0.75        33
           2       0.63      0.55      0.59        31

    accuracy                           0.63        95
   macro avg       0.63      0.63      0.63        95
weighted avg       0.63      0.63      0.63        95

Validation Accuracy (HOG 

## COMBINED FEATURE EXTRACTION

In [8]:
X_features = np.array([extract_combined_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)) for img in X])
X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [9]:
### TRAIN NON STACKING MODELS

model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_svm = SVC(kernel='linear', probability=True)
model_rf = RandomForestClassifier(n_estimators=50, random_state=42)
# Stacking Classifier
model_lr.fit(X_train, y_train)
model_svm.fit(X_train, y_train)
model_rf.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)

# Evaluate model
y_pred_lr_no = model_lr.predict(X_val)
y_pred_svm_no = model_svm.predict(X_val)
y_pred_rf_no = model_rf.predict(X_val)
y_pred_xgb_no = model_xgb.predict(X_val)

c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:01:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarnin

In [10]:
### NON STACKING MODELS ACCURACY

print(f"Validation Accuracy With Logistic Regression With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_lr_no) * 100:.2f}%")
print(classification_report(y_val, y_pred_lr_no))
print(f"Validation Accuracy With Support Vector Machine With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_svm_no) * 100:.2f}%")
print(classification_report(y_val, y_pred_svm_no))
print(f"Validation Accuracy With Random Forest With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf_no) * 100:.2f}%")
print(classification_report(y_val, y_pred_rf_no))
print(f"Validation Accuracy With XGBoosting With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_xgb_no) * 100:.2f}%")
print(classification_report(y_val, y_pred_xgb_no))

Validation Accuracy With Logistic Regression With Gamma Correction:
Accuracy: 67.37%
              precision    recall  f1-score   support

           0       0.56      0.58      0.57        31
           1       0.71      0.76      0.74        33
           2       0.75      0.68      0.71        31

    accuracy                           0.67        95
   macro avg       0.68      0.67      0.67        95
weighted avg       0.68      0.67      0.67        95

Validation Accuracy With Support Vector Machine With Gamma Correction:
Accuracy: 67.37%
              precision    recall  f1-score   support

           0       0.59      0.52      0.55        31
           1       0.73      0.82      0.77        33
           2       0.68      0.68      0.68        31

    accuracy                           0.67        95
   macro avg       0.67      0.67      0.67        95
weighted avg       0.67      0.67      0.67        95

Validation Accuracy With Random Forest With Gamma Correction:
Acc

In [13]:
### TRAIN STACKING MODELS

# Train base models
base_models = [
    ('svm', SVC(kernel='linear', probability=True)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
]
# Meta-model
meta_model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
meta_model_svm = SVC(kernel='linear', probability=True)
meta_model_rf = RandomForestClassifier(n_estimators=50, random_state=42)
meta_model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Stacking Classifier
stacking_lr = StackingClassifier(estimators=base_models, final_estimator=meta_model_lr, cv=5)
stacking_svm = StackingClassifier(estimators=base_models, final_estimator=meta_model_svm, cv=5)
stacking_rf = StackingClassifier(estimators=base_models, final_estimator=meta_model_rf, cv=5)
stacking_xgb = StackingClassifier(estimators=base_models, final_estimator=meta_model_xgb, cv=5)
stacking_lr.fit(X_train, y_train)
stacking_svm.fit(X_train, y_train)
stacking_rf.fit(X_train, y_train)
stacking_xgb.fit(X_train, y_train)

# Evaluate model
y_pred_lr = stacking_lr.predict(X_val)
y_pred_svm = stacking_svm.predict(X_val)
y_pred_rf = stacking_rf.predict(X_val)
y_pred_xgb = stacking_xgb.predict(X_val)

c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:56:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:58:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Geovanka\Documents\Kuliaaaa\Semester 5\Computer Vision (ComV)\Python 11\environments\computer_vision\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:23] WARNING: C:\buildki

In [14]:
print(f"Validation Accuracy Stacking With Logistic Regression With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_lr) * 100:.2f}%")
print(classification_report(y_val, y_pred_lr))
print(f"Validation Accuracy Stacking With Support Vector Machine With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_svm) * 100:.2f}%")
print(classification_report(y_val, y_pred_svm))
print(f"Validation Accuracy Stacking With Random Forest With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf) * 100:.2f}%")
print(classification_report(y_val, y_pred_rf))
print(f"Validation Accuracy Stacking With XGBoosting With Gamma Correction:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_xgb) * 100:.2f}%")
print(classification_report(y_val, y_pred_xgb))

Validation Accuracy Stacking With Logistic Regression With Gamma Correction:
Accuracy: 73.68%
              precision    recall  f1-score   support

           0       0.81      0.55      0.65        31
           1       0.73      0.91      0.81        33
           2       0.70      0.74      0.72        31

    accuracy                           0.74        95
   macro avg       0.75      0.73      0.73        95
weighted avg       0.75      0.74      0.73        95

Validation Accuracy Stacking With Support Vector Machine With Gamma Correction:
Accuracy: 73.68%
              precision    recall  f1-score   support

           0       0.77      0.65      0.70        31
           1       0.74      0.85      0.79        33
           2       0.71      0.71      0.71        31

    accuracy                           0.74        95
   macro avg       0.74      0.73      0.73        95
weighted avg       0.74      0.74      0.73        95

Validation Accuracy Stacking With Random Forest

In [18]:
### SAVE BEST MODELS

file = open('StackingSVMWithGamma74.pkl', 'wb')
pickle.dump(stacking_svm, file)
file.close()

file = open('StackingLRWithGamma74.pkl', 'wb')
pickle.dump(stacking_lr, file)
file.close()